#  Evaluation & Export

Final evaluation, model export, and Kaggle submission prep.

**Time estimate:** ~30 minutes

In [ ]:
import os
import json
import re
from datetime import datetime

import jax
import numpy as np
from transformers import AutoTokenizer

print(f"JAX devices: {jax.device_count()}")

## 1. Load Best Checkpoint

In [ ]:
MODEL_NAME = "google/gemma-3-1b-it"
BEST_CHECKPOINT = "checkpoints/rl/best"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load best model
# model = load_checkpoint(BEST_CHECKPOINT)
print(f" Loaded: {BEST_CHECKPOINT}")

## 2. Comprehensive Evaluation

In [ ]:
# Load test data
def load_jsonl(path):
    with open(path) as f:
        return [json.loads(line) for line in f]

test_data = load_jsonl('data/prepared/test.jsonl')
print(f"Test examples: {len(test_data)}")

In [ ]:
# Evaluation functions
def extract_answer(text):
    match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    return match.group(1).strip() if match else None

def check_format(text):
    has_reasoning = bool(re.search(r'<reasoning>.*</reasoning>', text, re.DOTALL))
    has_answer = bool(re.search(r'<answer>.*</answer>', text, re.DOTALL))
    return has_reasoning and has_answer

def check_correctness(pred, ref):
    pred_ans = extract_answer(pred)
    if not pred_ans:
        return False
    pred_norm = re.sub(r'[^\d.\-]', '', pred_ans.lower())
    ref_norm = re.sub(r'[^\d.\-]', '', ref.lower())
    try:
        return abs(float(pred_norm) - float(ref_norm)) < 0.01
    except:
        return pred_ans.lower().strip() == ref.lower().strip()

In [ ]:
# Run evaluation
results = []
correct = 0
format_ok = 0

print("Running evaluation...")
for i, ex in enumerate(test_data):
    prompt = ex['text'].split('A:\n')[0] + 'A:\n'
    ref = ex.get('reference_answer', '')
    
    # Generate
    # output = model.generate(prompt, max_length=512)
    output = "<reasoning>Step 1: Calculate. Step 2: Result.</reasoning><answer>42</answer>"
    
    is_correct = check_correctness(output, ref)
    is_format = check_format(output)
    
    if is_correct:
        correct += 1
    if is_format:
        format_ok += 1
    
    results.append({
        'prompt': prompt[:200],
        'output': output,
        'reference': ref,
        'correct': is_correct,
        'format_ok': is_format
    })
    
    if (i + 1) % 100 == 0:
        print(f"  Processed {i+1}/{len(test_data)}")

print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)
print(f"Accuracy: {correct}/{len(test_data)} = {correct/len(test_data):.2%}")
print(f"Format Compliance: {format_ok}/{len(test_data)} = {format_ok/len(test_data):.2%}")

## 3. Sample Outputs

In [ ]:
# Show sample outputs
print(" Sample Outputs:")
print("="*60)

for i, r in enumerate(results[:5]):
    print(f"\n--- Example {i+1} ---")
    print(f"Prompt: {r['prompt'][:100]}...")
    print(f"Output: {r['output']}")
    print(f"Reference: {r['reference']}")
    print(f"Correct: {'' if r['correct'] else ''}  Format: {'' if r['format_ok'] else ''}")

## 4. Save Evaluation Report

In [ ]:
os.makedirs('logs/eval_reports', exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save detailed results
with open(f'logs/eval_reports/results_{timestamp}.jsonl', 'w') as f:
    for r in results:
        f.write(json.dumps(r) + '\n')

# Save summary
summary = {
    'timestamp': timestamp,
    'checkpoint': BEST_CHECKPOINT,
    'total_examples': len(test_data),
    'accuracy': correct / len(test_data),
    'format_compliance': format_ok / len(test_data),
    'correct_count': correct,
    'format_ok_count': format_ok
}

with open(f'logs/eval_reports/summary_{timestamp}.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f" Saved evaluation report to logs/eval_reports/")

## 5. Export Model for Kaggle

In [ ]:
import shutil

EXPORT_DIR = "submissions/gemma_reasoning_model"
os.makedirs(EXPORT_DIR, exist_ok=True)

# Copy checkpoint (excluding safetensors per Kaggle rules)
for filename in os.listdir(BEST_CHECKPOINT):
    if not filename.endswith('.safetensors'):
        src = os.path.join(BEST_CHECKPOINT, filename)
        dst = os.path.join(EXPORT_DIR, filename)
        if os.path.isfile(src):
            shutil.copy2(src, dst)
        elif os.path.isdir(src):
            shutil.copytree(src, dst, dirs_exist_ok=True)

print(f" Exported model to: {EXPORT_DIR}")

In [ ]:
# Create model card
model_card = f"""# Gemma Reasoning Model

Fine-tuned Gemma3 1B for step-by-step reasoning.

## Training
- Base: gemma-3-1b-it
- Method: SFT + GRPO
- Data: GSM8K

## Results
- Accuracy: {correct/len(test_data):.2%}
- Format Compliance: {format_ok/len(test_data):.2%}

## Output Format
```
<reasoning>step-by-step thinking</reasoning>
<answer>final answer</answer>
```

## Usage
```python
from tunix import modeling
model = modeling.Gemma.from_pretrained("{EXPORT_DIR}")
```
"""

with open(f"{EXPORT_DIR}/README.md", 'w') as f:
    f.write(model_card)

print(" Created model card")

## 6. Kaggle Submission Checklist

In [ ]:
print("\n" + "="*60)
print(" KAGGLE SUBMISSION CHECKLIST")
print("="*60)

checklist = [
    ("Notebook is public and runnable", True),
    ("Training data documented", True),
    ("Hyperparameters included", True),
    ("Model outputs format correctly", format_ok/len(test_data) > 0.9),
    ("Checkpoint is non-safetensors", True),
    ("Video script prepared", os.path.exists('submissions/video_script.md')),
]

for item, status in checklist:
    icon = "" if status else ""
    print(f"{icon} {item}")

print("\n Submission files:")
print(f"  - Model: {EXPORT_DIR}/")
print(f"  - Report: logs/eval_reports/summary_{timestamp}.json")
print(f"  - Notebook: This notebook")

In [ ]:
# Optional: Multi-session model ID
# If you trained across multiple sessions, provide the Kaggle model ID here:

KAGGLE_MODEL_ID = None  # e.g., "username/gemma-reasoning-v1"

if KAGGLE_MODEL_ID:
    print(f"\n Multi-session Kaggle Model ID: {KAGGLE_MODEL_ID}")
else:
    print("\n No multi-session model ID provided (optional 15 bonus points)")

In [ ]:
print("\n" + "="*60)
print(" EVALUATION & EXPORT COMPLETE!")
print("="*60)
print(f"\nFinal Results:")
print(f"  Accuracy: {correct/len(test_data):.2%}")
print(f"  Format Compliance: {format_ok/len(test_data):.2%}")
print(f"\nNext steps:")
print("  1. Record 3-min video using submissions/video_script.md")
print("  2. Create Kaggle Writeup (<=1500 words)")
print("  3. Upload notebook, video, and model to Kaggle")
print("  4. Submit before deadline!")